In [1]:
import tensorflow as tf
import os
import IPython.display as display
from matplotlib import pyplot as plt
import h5py
import numpy as np
from tqdm import tqdm
import functools
%matplotlib inline 

2022-11-23 10:29:52.598223: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# root dir
dataset_dir  = './dataset/multi-object-datasets'

In [3]:
# parse clevr
dataset = 'clevr_with_masks'
dataset_file = 'clevr_with_masks_train.tfrecords'
dataset_path = os.path.join(dataset_dir,dataset,dataset_file)
IMAGE_SIZE = [240, 320]
MAX_NUM_ENTITIES = 11
COMPRESSION_TYPE = tf.io.TFRecordOptions.get_compression_type_string('GZIP')
BYTE_FEATURES = ['mask', 'image', 'color', 'material', 'shape', 'size']
features = {
    'image': tf.io.FixedLenFeature(IMAGE_SIZE+[3], tf.string),
    'mask': tf.io.FixedLenFeature([MAX_NUM_ENTITIES]+IMAGE_SIZE+[1], tf.string),
    'x': tf.io.FixedLenFeature([MAX_NUM_ENTITIES], tf.float32),
    'y': tf.io.FixedLenFeature([MAX_NUM_ENTITIES], tf.float32),
    'z': tf.io.FixedLenFeature([MAX_NUM_ENTITIES], tf.float32),
    'pixel_coords': tf.io.FixedLenFeature([MAX_NUM_ENTITIES, 3], tf.float32),
    'rotation': tf.io.FixedLenFeature([MAX_NUM_ENTITIES], tf.float32),
    'size': tf.io.FixedLenFeature([MAX_NUM_ENTITIES], tf.string),
    'material': tf.io.FixedLenFeature([MAX_NUM_ENTITIES], tf.string),
    'shape': tf.io.FixedLenFeature([MAX_NUM_ENTITIES], tf.string),
    'color': tf.io.FixedLenFeature([MAX_NUM_ENTITIES], tf.string),
    'visibility': tf.io.FixedLenFeature([MAX_NUM_ENTITIES], tf.float32),
}
raw_dataset = tf.data.TFRecordDataset(dataset_path)

def _decode(example_proto):
  # Parse the input `tf.Example` proto using the feature description dict above.
  single_example = tf.io.parse_single_example(example_proto, features)
  for k in BYTE_FEATURES:
    single_example[k] = tf.squeeze(tf.io.decode_raw(single_example[k], tf.uint8),
                                   axis=-1)
  return single_example

def dataset_loader(tfrecords_path, read_buffer_size=None, map_parallel_calls=None):
  """Read, decompress, and parse the TFRecords file.
  Args:
    tfrecords_path: str. Path to the dataset file.
    read_buffer_size: int. Number of bytes in the read buffer. See documentation
      for `tf.data.TFRecordDataset.__init__`.
    map_parallel_calls: int. Number of elements decoded asynchronously in
      parallel. See documentation for `tf.data.Dataset.map`.
  Returns:
    An unbatched `tf.data.TFRecordDataset`.
  """
  raw_dataset = tf.data.TFRecordDataset(
      tfrecords_path, compression_type=COMPRESSION_TYPE,
      buffer_size=read_buffer_size)
  return raw_dataset.map(_decode, num_parallel_calls=map_parallel_calls)

parsed_image_dataset = dataset_loader(dataset_path)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2022-11-23 10:29:54.774681: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-23 10:29:54.779840: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-23 10:29:54.850419: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-23 10:29:54.850987: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2070 with Max-Q Design computeCapability: 7.5
coreClock: 1.29GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 357.69GiB/s
2022-11-23 10:29:54.851007: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-11-23 10:29:54.898731: I tensorflow/stream_exe

In [4]:
# write clevr to h5
image_list = []
mask_list = []
x_list = []
y_list = []
z_list = []
pixel_coords_list = []
rotation_list = []
size_list = []
material_list = []
shape_list = []
color_list = []
visibility_list = []

h5_path = dataset_path = os.path.join(dataset_dir,dataset,dataset_file.split('.')[0]+'.h5')
with h5py.File(h5_path, 'w') as f:
    image_group = f.create_group('image')
    mask_group = f.create_group('mask')
    x_group = f.create_group('x')
    y_group = f.create_group('y')
    z_group = f.create_group('z')
    pixel_coords_group = f.create_group('pixel_coords')
    rotation_group = f.create_group('rotation')
    size_group = f.create_group('size')
    material_group = f.create_group('material')
    color_group = f.create_group('color')
    visibility_group = f.create_group('visibility')
    shape_group = f.create_group('shape')


    for idx, image_features in tqdm(enumerate(parsed_image_dataset)):
        key = str(idx).zfill(6)
        image_group[key] = image_features['image'].numpy()
        mask_group[key] = image_features['mask'].numpy()
        x_group[key] = image_features['x'].numpy()
        y_group[key] = image_features['y'].numpy()
        z_group[key] = image_features['z'].numpy()
        shape_group[key] = image_features['shape'].numpy()
        pixel_coords_group[key] = image_features['pixel_coords'].numpy()
        rotation_group[key] = image_features['rotation'].numpy()
        size_group[key] = image_features['size'].numpy()
        material_group[key] = image_features['material'].numpy()
        color_group[key] = image_features['color'].numpy()
        visibility_group[key] = image_features['visibility'].numpy()

2022-11-23 10:30:05.810120: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-11-23 10:30:05.829393: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz
100000it [54:59, 30.31it/s]


In [32]:
# parse multi_dsprites
dataset = 'multi_dsprites'
dataset_file = 'multi_dsprites_colored_on_grayscale.tfrecords'
dataset_path = os.path.join(dataset_dir,dataset,dataset_file)
IMAGE_SIZE = [64, 64]
MAX_NUM_ENTITIES = {
    'binarized': 4,
    'colored_on_grayscale': 6,
    'colored_on_colored': 5
}
COMPRESSION_TYPE = tf.io.TFRecordOptions.get_compression_type_string('GZIP')
BYTE_FEATURES = ['mask', 'image']

def feature_descriptions(max_num_entities, is_grayscale=False):
  """Create a dictionary describing the dataset features.
  Args:
    max_num_entities: int. The maximum number of foreground and background
      entities in each image. This corresponds to the number of segmentation
      masks and generative factors returned per scene.
    is_grayscale: bool. Whether images are grayscale. Otherwise they're assumed
      to be RGB.
  Returns:
    A dictionary which maps feature names to `tf.Example`-compatible shape and
    data type descriptors.
  """

  num_channels = 1 if is_grayscale else 3
  return {
      'image': tf.io.FixedLenFeature(IMAGE_SIZE+[num_channels], tf.string),
      'mask': tf.io.FixedLenFeature(IMAGE_SIZE+[max_num_entities, 1], tf.string),
      'x': tf.io.FixedLenFeature([max_num_entities], tf.float32),
      'y': tf.io.FixedLenFeature([max_num_entities], tf.float32),
      'shape': tf.io.FixedLenFeature([max_num_entities], tf.float32),
      'color': tf.io.FixedLenFeature([max_num_entities, num_channels], tf.float32),
      'visibility': tf.io.FixedLenFeature([max_num_entities], tf.float32),
      'orientation': tf.io.FixedLenFeature([max_num_entities], tf.float32),
      'scale': tf.io.FixedLenFeature([max_num_entities], tf.float32),
  }

def _decode(example_proto, features):
  # Parse the input `tf.Example` proto using a feature description dictionary.
  single_example = tf.io.parse_single_example(example_proto, features)
  for k in BYTE_FEATURES:
    single_example[k] = tf.squeeze(tf.io.decode_raw(single_example[k], tf.uint8),
                                   axis=-1)
  # To return masks in the canonical [entities, height, width, channels] format,
  # we need to transpose the tensor axes.
  single_example['mask'] = tf.transpose(single_example['mask'], [2, 0, 1, 3])
  return single_example

def dataset_loader(tfrecords_path, dataset_variant, read_buffer_size=None,
            map_parallel_calls=None):
  """Read, decompress, and parse the TFRecords file.
  Args:
    tfrecords_path: str. Path to the dataset file.
    dataset_variant: str. One of ['binarized', 'colored_on_grayscale',
      'colored_on_colored']. This is used to identify the maximum number of
      entities in each scene. If an incorrect identifier is passed in, the
      TFRecords file will not be read correctly.
    read_buffer_size: int. Number of bytes in the read buffer. See documentation
      for `tf.data.TFRecordDataset.__init__`.
    map_parallel_calls: int. Number of elements decoded asynchronously in
      parallel. See documentation for `tf.data.Dataset.map`.
  Returns:
    An unbatched `tf.data.TFRecordDataset`.
  """
  if dataset_variant not in MAX_NUM_ENTITIES:
    raise ValueError('Invalid `dataset_variant` provided. The supported values'
                     ' are: {}'.format(list(MAX_NUM_ENTITIES.keys())))
  max_num_entities = MAX_NUM_ENTITIES[dataset_variant]
  is_grayscale = dataset_variant == 'binarized'
  raw_dataset = tf.data.TFRecordDataset(
      tfrecords_path, compression_type=COMPRESSION_TYPE,
      buffer_size=read_buffer_size)
  features = feature_descriptions(max_num_entities, is_grayscale)
  partial_decode_fn = functools.partial(_decode, features=features)
  return raw_dataset.map(partial_decode_fn,
                         num_parallel_calls=map_parallel_calls)



parsed_image_dataset = dataset_loader(dataset_path,'colored_on_grayscale')

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [33]:
# write multi_dsprites to h5
image_list = []
mask_list = []
x_list = []
y_list = []
shape_list = []
color_list = []
visibility_list = []
orientation_list = []
scale_list = []

h5_path = dataset_path = os.path.join(dataset_dir,dataset,dataset_file.split('.')[0]+'.h5')
with h5py.File(h5_path, 'w') as f:
    image_group = f.create_group('image')
    mask_group = f.create_group('mask')
    x_group = f.create_group('x')
    y_group = f.create_group('y')
    shape_group = f.create_group('shape')
    color_group = f.create_group('color')
    visibility_group = f.create_group('visibility')
    orientation_group = f.create_group('orientation')
    scale_group = f.create_group('scale')

    for idx, image_features in tqdm(enumerate(parsed_image_dataset)):
        if idx >80000:
            break
        key = str(idx).zfill(6)
        image_group[key] = image_features['image'].numpy()
        mask_group[key] = image_features['mask'].numpy()
        x_group[key] = image_features['x'].numpy()
        y_group[key] = image_features['y'].numpy()
        shape_group[key] = image_features['shape'].numpy()
        color_group[key] = image_features['color'].numpy()
        visibility_group[key] = image_features['visibility'].numpy()
        orientation_group[key] = image_features['orientation'].numpy()
        scale_group[key] = image_features['scale'].numpy()

80001it [03:51, 345.53it/s]


In [28]:
# parse tetrominoes
dataset = 'tetrominoes'
dataset_file = 'tetrominoes_train.tfrecords'
dataset_path = os.path.join(dataset_dir,dataset,dataset_file)
COMPRESSION_TYPE = tf.io.TFRecordOptions.get_compression_type_string('GZIP')
IMAGE_SIZE = [35, 35]
# The maximum number of foreground and background entities in the provided
# dataset. This corresponds to the number of segmentation masks returned per
# scene.
MAX_NUM_ENTITIES = 4
BYTE_FEATURES = ['mask', 'image']

# Create a dictionary mapping feature names to `tf.Example`-compatible
# shape and data type descriptors.
features = {
    'image': tf.io.FixedLenFeature(IMAGE_SIZE+[3], tf.string),
    'mask': tf.io.FixedLenFeature([MAX_NUM_ENTITIES]+IMAGE_SIZE+[1], tf.string),
    'x': tf.io.FixedLenFeature([MAX_NUM_ENTITIES], tf.float32),
    'y': tf.io.FixedLenFeature([MAX_NUM_ENTITIES], tf.float32),
    'shape': tf.io.FixedLenFeature([MAX_NUM_ENTITIES], tf.float32),
    'color': tf.io.FixedLenFeature([MAX_NUM_ENTITIES, 3], tf.float32),
    'visibility': tf.io.FixedLenFeature([MAX_NUM_ENTITIES], tf.float32),
}

def _decode(example_proto):
  # Parse the input `tf.Example` proto using the feature description dict above.
  single_example = tf.io.parse_single_example(example_proto, features)
  for k in BYTE_FEATURES:
    single_example[k] = tf.squeeze(tf.io.decode_raw(single_example[k], tf.uint8),
                                   axis=-1)
  return single_example


def dataset_loader(tfrecords_path, read_buffer_size=None, map_parallel_calls=None):
  """Read, decompress, and parse the TFRecords file.
  Args:
    tfrecords_path: str. Path to the dataset file.
    read_buffer_size: int. Number of bytes in the read buffer. See documentation
      for `tf.data.TFRecordDataset.__init__`.
    map_parallel_calls: int. Number of elements decoded asynchronously in
      parallel. See documentation for `tf.data.Dataset.map`.
  Returns:
    An unbatched `tf.data.TFRecordDataset`.
  """
  raw_dataset = tf.data.TFRecordDataset(
      tfrecords_path, compression_type=COMPRESSION_TYPE,
      buffer_size=read_buffer_size)
  return raw_dataset.map(_decode, num_parallel_calls=map_parallel_calls)

parsed_image_dataset = dataset_loader(dataset_path)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [29]:
# write tetrominoes to h5
image_list = []
mask_list = []
x_list = []
y_list = []
shape_list = []
color_list = []
visibility_list = []


h5_path = dataset_path = os.path.join(dataset_dir,dataset,dataset_file.split('.')[0]+'.h5')
with h5py.File(h5_path, 'w') as f:
    image_group = f.create_group('image')
    mask_group = f.create_group('mask')
    x_group = f.create_group('x')
    y_group = f.create_group('y')
    shape_group = f.create_group('shape')
    color_group = f.create_group('color')
    visibility_group = f.create_group('visibility')


    for idx, image_features in tqdm(enumerate(parsed_image_dataset)):
        if idx >80000:
            break
        key = str(idx).zfill(6)
        image_group[key] = image_features['image'].numpy()
        mask_group[key] = image_features['mask'].numpy()
        x_group[key] = image_features['x'].numpy()
        y_group[key] = image_features['y'].numpy()
        shape_group[key] = image_features['shape'].numpy()
        color_group[key] = image_features['color'].numpy()
        visibility_group[key] = image_features['visibility'].numpy()

80001it [02:30, 530.18it/s]
